  # 0. Bibliotek och data
  Biblioteket `sqlite3` krävs för att skapa funktionen `Nyrad()` nedan. SLOSH-datafilen läses in.

In [6]:
import sqlite3
import pandas as pd

In [7]:
datafil = r'\\win.su.se\dfs\common\stressforskning.su.se\SLOSH-data\SLOSH och AMU\SLOSH\Enkätdata\SLOSH (Huvudfil, t.o.m. 2018)\Test\Slosh_all_amuar_18.xlsx'

df = pd.read_excel(datafil)

# 1. Definiera funktionen  
Funktionen `Nyrad()` tar (namnet på) en tabell, en tuple med kolumnnamn samt en tuple med kolumnvärden. Den skapar en rad av denna information och försöker lägga in den i databasen. Om den lyckas kommer den SQL-kod som används för att skapa raden att noteras i en text-fil. Denna text-fil ska tjäna dels som en backup men även som en logg. Skulle något hända med databasen går det enkelt att bara köra text-filen så återskapas allting.  
  Ifall funktionen inte lyckas lägga in en rad i databasen, kanske p.g.a. att "foregin key" inte respekteras, kommer ett felmeddelande konstatera detta. För mer information om hur SQLite används i python: https://pynative.com/python-sqlite-insert-into-table/.

In [8]:
def Nyrad(tabell, kolumner, värden):    
    try:
        # Koppla databasen
        sqliteConnection = sqlite3.connect('lala.db')

        # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
        cursor = sqliteConnection.cursor()

        print("Successfully Connected to SQLite")

        # Denna query ska skrivas i SQL och körs av python mot databasen
        sqlite_insert_query = 'INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden)

        # Först måste 'foreign key constraint' slås på, i SQLite är detta avstängt som default. 
        cursor.execute('PRAGMA foreign_keys = 1')

        # Nu kan raden läggas in
        cursor.execute(sqlite_insert_query)

        # commit:a till databasen
        sqliteConnection.commit()
        print("Rad inlagd i tabellen {}".format(tabell), cursor.rowcount)

        # Stäng
        cursor.close()
        
        # När en rad lagts till ska SQL-kommandot sparas i en textfil.
        text = open("SQL-logg.txt", "a", encoding = 'UTF-8')
        text.write('INSERT INTO {} {} VALUES {};\n \n'.format(tabell, kolumner, värden))
        text.close()

    except sqlite3.Error as error:
        print("Failed to insert data into sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")
    

# 2.Transkribera item  
Här transkriberas ett item i taget. Den transkriberade informationen kommer att användas för att skapa rader i flera olika tabeller. Exempelvis kommer `kodlista` i tre olika tabeller. Börja med att se efter ifall en given variabel förekommer i working, non-working eller båda. 

In [19]:
totvar = len(df.columns)

# Koppla databasen
sqliteConnection = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = sqliteConnection.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen
sqlite_insert_query = 'SELECT enkättext FROM Enkättexter'

# Nu kan raden läggas in
cursor.execute(sqlite_insert_query)

dokvar = len(cursor.fetchall())

# Stäng
cursor.close()

print('Andel dokumenterad data: {}%'.format(round(100*dokvar/totvar, 2)))

Andel dokumenterad data: 13.76%


In [26]:
variabel = 'promoted_2'

In [29]:
r = 'r_{}'.format(variabel[-1])

frek_nw = df.loc[df[r] == 1, variabel].value_counts()

frek_w = df.loc[df[r] == 2, variabel].value_counts()

if len(frek_w) == 0:
    print('Variabeln {} förekommer ej i working'.format(variabel))
    
elif len(frek_nw) == 0:
    print('Variabeln {} förekommer ej i non-working'.format(variabel))
    
else:
    print('Variabeln {} förekommer i working och i non-working'.format(variabel))

    

Variabeln promoted_2 förekommer ej i non-working


Givet testet ovan går det att transkribera resten av informationen. Notera att ifall variabeln förekommer i både working och non-working måste det göras två insättningar relationsdatabasen. 

In [ ]:
enkät = ''
beskrivning = ''
itemnr = ''
enkättext = ''

kodlista = ''
kodsvar = {}

# 4. Infoga rader i databasen  
Nu kommer den transkriberade informationen att användas tillsamman med funktionen `Nyrad()` för var och en av tabellerna i databasen.

## C: Kodlistor

In [ ]:
tabell = 'C: Kodlistor'

kolumner = ('KODLISTA')

värden = (kodlista)

In [ ]:
Nyrad(tabell, kolumner, värden)

## G: Enkättexter

In [ ]:
tabell = 'G: Enkättexter'

kolumner = ('ENKÄTTEXT')

värden = (enkättext)

In [ ]:
Nyrad(tabell, kolumner, värden)

## H: Svarstexter

In [ ]:
tabell = 'H: Svarstexter'

kolumner = ('SVARSTEXT')

värden = (svarstext)

In [ ]:
Nyrad(tabell, kolumner, värden)

## D: Kodlistor_stor

In [ ]:
tabell = 'D: Kodlistor_stor'

kolumner = ('K_ID', 'KODLISTA', 'KOD', 'SVARSTEXT')

In [2]:
for i in kodsvar.keys():
    värden = (kodlista, i, kodsvar[i])
    Nyrad(tabell, kolumner, värden)

<function dict.keys>

## B: Variabler

In [ ]:
tabell = 'B: Variabler'

kolumner = ('VARIABEL', 'ENKÄT', 'BESKRIVNING', 'ITEMNR', 'ENKÄTTEXT', 'KODLISTA')

värden = (variabel, enkät, beskrivning, itemnr, enkättext, kodlista)

In [ ]:
Nyrad(tabell, kolumner, värden)